In [1]:
import pandas as pd
import numpy as np
import json
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG, display
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
import pandas as pd
import os
import itertools
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp
from multiprocessing import Pool

from tqdm import tqdm 
import cobra
from itertools import chain, combinations
import sys
import multiprocessing
from functools import partial
tqdm.pandas()
# sys.path.append('../Code/')
sys.path.append('../')

from common import *

In [33]:
#input
num = 50
cut_off = 0.3
cut_off_retrosynthesis_path = f'../../../Data_retrosynthesis/not_lipid_filter20percent/top{num}_{cut_off}_add_no_ec_re/'
cut_off_path = f'../../../Results/not_lipid_filter20percent/top{num}_{cut_off}_add_no_ec_re/'
not_lipid_rxndb_path = cut_off_retrosynthesis_path + 'RXNdb_top50_0.3/'
not_lipid_YMDB_fail_met_smile_uptake_file = cut_off_path + f'YMDB_fail_met_smile_top{num}_{cut_off}.pickle'
# total_met_inchikey0_file = '../../../Results/analysis/total_met_inchikey0.pickle'
total_met_inchikey0_file = '../../../Results/analysis/total_met_inchikey0_filter.pickle'
YMDB_fail_met_smile_file = cut_off_path + f'YMDB_fail_met_smile_top{num}_{cut_off}.pickle'
uptake_met_path = '../../../Data/ymdb/ymdb_uptake.csv'
rxndb_path = cut_off_retrosynthesis_path + f'RXNdb_top{num}_{cut_off}/'
not_lipid_target_smiles_filter20percent_else_path = '../../../Results/not_lipid_filter20percent/target_smiles_not_lipid_filter20percent_else.pickle'

#output
not_lipid_rxndb_inteme_drop_duplicate_path = cut_off_path + f'RXNdb_all_top50_0.3_all.csv'
not_lipid_fail_target_rxndb_path = cut_off_retrosynthesis_path + 'RXNdb_fail_target/'
not_lipid_rxndb_inteme_only_one_new_path = cut_off_path + 'RXNdb_inteme_only_one_new_top50_drop_unvalid.csv'
not_lipid_rxndb_inteme_only_one_new_target_path = cut_off_path + 'RXNdb_inteme_only_one_new_target_top50.csv'
sink0_path = cut_off_path + 'sink0_new_met_only_new_one_top50.pickle'

In [3]:
if not os.path.exists(not_lipid_fail_target_rxndb_path):
    os.makedirs(not_lipid_fail_target_rxndb_path)

In [4]:
def process_file_mer(file,rxndb_path,inchikey0,rxndb_drop_path):
    with open(rxndb_path + file, 'r') as f:
        rxndb = json.load(f)    
    remove = []
    for k,v in rxndb.items():
        
        productsmile = v['productsmile'].split('.')
        if any([smiles2inchikey0(p) in inchikey0  for p in productsmile]):
            pass
        else:
            remove.append(k)
    for k in remove:
        rxndb.pop(k)
    with open(rxndb_drop_path + file, 'w') as f:
        json.dump(rxndb, f,indent=4)

def drop_rxndb_mer(rxndb_path, rxndb_drop_path, inchikey0,num_processes=60):
    files = os.listdir(rxndb_path)
    process_file_partial = partial(process_file_mer,rxndb_path=rxndb_path,inchikey0=inchikey0,rxndb_drop_path=rxndb_drop_path)
    with mp.Pool(num_processes) as p:
        list(tqdm(p.imap_unordered(process_file_partial, files), total=len(files)))

In [5]:
# not_lipid_YMDB_fail_met_smile = pickle.load(open(not_lipid_YMDB_fail_met_smile_uptake_file, 'rb'))
not_lipid_YMDB_fail_met_smile = pickle.load(open(not_lipid_target_smiles_filter20percent_else_path, 'rb'))
not_lipid_YMDB_fail_met_inchikey0 = [smiles2inchikey0(smile) for smile in not_lipid_YMDB_fail_met_smile ]

In [6]:
drop_rxndb_mer(not_lipid_rxndb_path, not_lipid_fail_target_rxndb_path, not_lipid_YMDB_fail_met_inchikey0,num_processes=50)

100%|██████████| 29161/29161 [04:38<00:00, 104.61it/s]


In [7]:
def process_file_drop_duplicate(file,rxndb_path):
    try:
        with open(rxndb_path + file, 'r') as f:
            rxndb = json.load(f)    
    
        if len(rxndb) != 0:
            rxndb_df = pd.DataFrame(rxndb).transpose()
            
            rxndb_df['reactant_inchikey0'] = rxndb_df['reactant_smile'].apply(lambda x: '.'.join(sorted([smiles2inchikey0(i) for i in (x.split('.'))])))
            rxndb_df['product_inchikey0'] = rxndb_df['productsmile'].apply(lambda x: '.'.join(sorted([smiles2inchikey0(i) for i in (x.split('.'))])))
            # rxndb_df['product_inchikey0'] = rxndb_df['product_inchikey0'].apply(lambda x: '.'.join(sorted(x.split('.'))))
            # rxndb_df['reactant_inchikey0'] = rxndb_df['reactant_inchikey0'].apply(lambda x: '.'.join(sorted(x.split('.'))))
            rxndb = rxndb_df.to_dict(orient='index')
            with open(rxndb_path + file, 'w') as f:
                json.dump(rxndb, f,indent=4)
    except:
        print(file)
        pass

def process_file_drop_duplicate_parallel(rxndb_path,num_processes=50):
    files = os.listdir(rxndb_path)
#     files = ['RXNdb_14838.json','RXNdb_7284.json','RXNdb_10683.json','RXNdb_10627.json',
# 'RXNdb_17365.json','RXNdb_13669.json','RXNdb_14857.json','RXNdb_15735.json','RXNdb_5346.json','RXNdb_7276.json']
    process_file_partial = partial(process_file_drop_duplicate,rxndb_path=rxndb_path)
    with mp.Pool(num_processes) as p:
        list(tqdm(p.imap_unordered(process_file_partial, files), total=len(files)))

In [11]:
def drop_duplicate_rxndb(not_lipid_rxndb_inteme_drop_duplicate_path,rxndb_path):
    files = os.listdir(rxndb_path) 
    with open(rxndb_path + files[0], 'r') as f:
        rxndb_total = json.load(f)
        rxndb_total_df = pd.DataFrame(rxndb_total).transpose()
    for file in tqdm(files[1:],total=len(files[1:])):
        with open(rxndb_path + file, 'r') as f:
            rxndb = json.load(f) 

        rxndb_df = pd.DataFrame(rxndb).transpose()
        rxndb_total_df = pd.concat([rxndb_total_df,rxndb_df],axis=0)
    rxndb_total_df = rxndb_total_df.drop_duplicates(subset=['reactant_inchikey0', 'product_inchikey0'], keep='first').reset_index(drop=True)
    rxndb_total_df.to_csv(not_lipid_rxndb_inteme_drop_duplicate_path )
    print(rxndb_total_df.shape)

def drop_duplicate_rxndb(not_lipid_rxndb_inteme_drop_duplicate_path, rxndb_path):
    files = os.listdir(rxndb_path) 
    all_data = []  # 用列表临时存储每个文件的数据
    for file in tqdm(files, total=len(files), desc="Processing files"):
        with open(os.path.join(rxndb_path, file), 'r') as f:  # 使用 os.path.join 更稳健
            rxndb = json.load(f)
        rxndb_df = pd.DataFrame(rxndb).transpose()
        all_data.append(rxndb_df)  # 将每个 DataFrame 添加到列表中
    # 一次性合并所有 DataFrame
    rxndb_total_df = pd.concat(all_data, axis=0)
    # 去重
    rxndb_total_df = rxndb_total_df.drop_duplicates(subset=['reactant_inchikey0', 'product_inchikey0'], keep='first').reset_index(drop=True)
    # 保存到 CSV 文件
    rxndb_total_df.to_csv(not_lipid_rxndb_inteme_drop_duplicate_path, index=False)  # index=False 避免多余索引

    # 打印最终 DataFrame 的行列数
    print(f"Final DataFrame shape: {rxndb_total_df.shape}")


In [9]:
process_file_drop_duplicate_parallel(not_lipid_fail_target_rxndb_path,num_processes=30)

  4%|▍         | 1308/29161 [00:13<11:19, 41.01it/s] 

100%|██████████| 29161/29161 [12:53<00:00, 37.72it/s] 


In [12]:
drop_duplicate_rxndb(not_lipid_rxndb_inteme_drop_duplicate_path,not_lipid_fail_target_rxndb_path)

Processing files:   0%|          | 0/29161 [00:00<?, ?it/s]

Processing files: 100%|██████████| 29161/29161 [06:39<00:00, 72.92it/s] 


Final DataFrame shape: (978830, 11)


In [33]:
# total_inchikey0 = load_pickle(total_met_inchikey0_file)
# not_lipid_target_smiles_filter20percent_else_path = '../../../Results/not_lipid_filter20percent/target_smiles_not_lipid_filter20percent_else.pickle'
# not_lipid_target_smiles_filter20percent_else = load_pickle(not_lipid_target_smiles_filter20percent_else_path)
# filter_inchi = [smiles2inchikey0(x) for x in not_lipid_target_smiles_filter20percent_else]
# for x in filter_inchi:
#     if x in total_inchikey0:
#         print(x)

# rxndb_total_df = pd.read_csv(not_lipid_rxndb_inteme_drop_duplicate_path)
# print(rxndb_total_df.shape)
# rxndb_total_df['new_num'] = None
# rxndb_total_df['reactant_inchikey0'] = rxndb_total_df['reactant_inchikey0'].apply(lambda x: x.split('.'))
# rxndb_total_df['product_inchikey0'] = rxndb_total_df['product_inchikey0'].apply(lambda x: x.split('.'))
# rxndb_total_df['new_num'] = rxndb_total_df['product_inchikey0'].apply(lambda x: sum([1 for i in x if i not in total_inchikey0]))
# rxndb_total_df = rxndb_total_df[rxndb_total_df['new_num'] == 0]
# print(rxndb_total_df.shape)
# rxndb_total_df['new_num'] = rxndb_total_df['reactant_inchikey0'].apply(lambda x: sum([1 for i in x if i in filter_inchi ]))
# rxndb_total_df = rxndb_total_df[rxndb_total_df['new_num'] > 0]
# print(rxndb_total_df.shape)
# rxndb_total_df['new_num'] = rxndb_total_df['reactant_inchikey0'].apply(lambda x: sum([1 for i in x if i not in total_inchikey0 ]))
# rxndb_total_df = rxndb_total_df[rxndb_total_df['new_num'] == 1]
# print(rxndb_total_df.shape)
# rxndb_total_df['product_inchikey0'] = rxndb_total_df['product_inchikey0'].apply(lambda x: ('.').join(x))
# rxndb_total_df.head()

(1130, 13)
(1130, 13)
(1130, 13)


,Unnamed: 0,EC number,rule,templateID,templateSubstrate,rxn_smiles_basic,rxn_smiles_final,reactant_smile,productsmile,similarity,reactant_inchikey0,product_inchikey0,new_num
18,18,3.1.3.25,[C:1]-[OH;D1;+0:2].[O;-;D1;H0:4]-[P;H0;D4;+0:3...,MNXR151315_reverse,['O[C@H]1[C@H](O)[C@@H](O)[C@H](O)[C@@H](O)[C@...,O.O=P([O-])([O-])OC1C(O)C(O)C(OP(=O)([O-])[O-]...,O.O=P([O-])([O-])OC1C(O)C(O)C(OP(=O)([O-])[O-]...,O.O=P([O-])([O-])OC1C(O)C(O)C(OP(=O)([O-])[O-]...,O=P([O-])([O-])OC1C(O)C(O)C(O)C(O)C1O.O=P([O-]...,"[0.6296296296, 1]","[PELZSPZCXGTUMR, XLYOFNOQVPJJNP]","[INAPMGSXUVUWAF, NBIIXXVUZAFLBC]",1
19,19,3.1.3.25,[C:1]-[OH;D1;+0:2].[O;-;D1;H0:4]-[P;H0;D4;+0:3...,MNXR151315_reverse,['O[C@H]1[C@H](O)[C@@H](O)[C@H](O)[C@@H](O)[C@...,O.O=P([O-])([O-])OC1C(O)C(O)C(O)C(O)C1OP(=O)([...,O.O=P([O-])([O-])OC1C(O)C(O)C(O)C(O)C1OP(=O)([...,O.O=P([O-])([O-])OC1C(O)C(O)C(O)C(O)C1OP(=O)([...,O=P([O-])([O-])OC1C(O)C(O)C(O)C(O)C1O.O=P([O-]...,"[0.6296296296, 1]","[MCKAJXMRULSUKI, XLYOFNOQVPJJNP]","[INAPMGSXUVUWAF, NBIIXXVUZAFLBC]",1
28,28,3.1.3.25,[C:1]-[OH;D1;+0:2].[O;-;D1;H0:4]-[P;H0;D4;+0:3...,MNXR151315_reverse,['O[C@H]1[C@H](O)[C@@H](O)[C@H](O)[C@@H](O)[C@...,O.O=P([O-])([O-])OC1C(O)C(OP(=O)([O-])[O-])C(O...,O.O=P([O-])([O-])OC1C(O)C(OP(=O)([O-])[O-])C(O...,O.O=P([O-])([O-])OC1C(O)C(OP(=O)([O-])[O-])C(O...,O=P([O-])([O-])OC1C(O)C(O)C(OP(=O)([O-])[O-])C...,"[0.6071428571, 1]","[XLYOFNOQVPJJNP, ZAWIXNGTTZTBKV]","[MMWCIQZXVOZEGG, NBIIXXVUZAFLBC]",1
112,112,3.1.3.25,[C:1]-[OH;D1;+0:2].[O;-;D1;H0:4]-[P;H0;D4;+0:3...,MNXR151315_reverse,['O[C@H]1[C@H](O)[C@@H](O)[C@H](O)[C@@H](O)[C@...,O.O=P([O-])([O-])OCC1OC(OC2(CO)OC(CO)C(O)C2O)C...,O.O=P([O-])([O-])OCC1OC(OC2(CO)OC(CO)C(O)C2O)C...,O.O=P([O-])([O-])OCC1OC(OC2(CO)OC(CO)C(O)C2O)C...,OCC1OC(OC2(CO)OC(CO)C(O)C2O)C(O)C(O)C1O.O=P([O...,"[0.4857142857, 1]","[WQQSIXKPRAUZJL, XLYOFNOQVPJJNP]","[CZMRCDWAGMRECN, NBIIXXVUZAFLBC]",1
125,125,3.1.3.25,[C:1]-[OH;D1;+0:2].[O;-;D1;H0:4]-[P;H0;D4;+0:3...,MNXR151315_reverse,['O[C@H]1[C@H](O)[C@@H](O)[C@H](O)[C@@H](O)[C@...,O.O=P([O-])([O-])OC1(CO)OC(CO)C(O)C1O>>OCC1OC(...,O.O=P([O-])([O-])OC1(CO)OC(CO)C(O)C1O>>OCC1OC(...,O.O=P([O-])([O-])OC1(CO)OC(CO)C(O)C1O,OCC1OC(O)(CO)C(O)C1O.O=P([O-])([O-])O,"[0.48484848480000003, 1]","[PMTUDJVZIGZBIX, XLYOFNOQVPJJNP]","[NBIIXXVUZAFLBC, RFSUNEUAIZKAJO]",1


In [34]:
def choose_only_one_new(not_lipid_rxndb_inteme_drop_duplicate_path,total_inchikey0,not_lipid_rxndb_inteme_only_one_new_path):
    rxndb_total_df = pd.read_csv(not_lipid_rxndb_inteme_drop_duplicate_path)
    rxndb_total_df['new_num'] = None
    rxndb_total_df['reactant_inchikey0'] = rxndb_total_df['reactant_inchikey0'].apply(lambda x: x.split('.'))
    rxndb_total_df['new_num'] = rxndb_total_df['reactant_inchikey0'].apply(lambda x: sum([1 for i in x if i not in total_inchikey0 ]))
    rxndb_total_df = rxndb_total_df[rxndb_total_df['new_num'] == 1]
    rxndb_total_df.to_csv(not_lipid_rxndb_inteme_only_one_new_path)

def choose_only_one_new(not_lipid_rxndb_inteme_drop_duplicate_path,not_lipid_target_smiles_filter20percent_else_path,total_inchikey0,not_lipid_rxndb_inteme_only_one_new_path):
    rxndb_total_df = pd.read_csv(not_lipid_rxndb_inteme_drop_duplicate_path)
    not_lipid_target_smiles_filter20percent_else = load_pickle(not_lipid_target_smiles_filter20percent_else_path)
    filter_inchi = [smiles2inchikey0(x) for x in not_lipid_target_smiles_filter20percent_else]
    rxndb_total_df['reactant_inchikey0'] = rxndb_total_df['reactant_inchikey0'].apply(lambda x: x.split('.'))
    rxndb_total_df['product_inchikey0'] = rxndb_total_df['product_inchikey0'].apply(lambda x: x.split('.'))
    print(rxndb_total_df.shape)
    tqdm.pandas()
    rxndb_total_df['new_num'] = rxndb_total_df['product_inchikey0'].progress_apply(lambda x: sum([1 for i in x if i not in total_inchikey0]))
    rxndb_total_df = rxndb_total_df[rxndb_total_df['new_num'] == 0]
    print(rxndb_total_df.shape)
    rxndb_total_df['new_num'] = rxndb_total_df['reactant_inchikey0'].progress_apply(lambda x: sum([1 for i in x if i in filter_inchi ]))
    rxndb_total_df = rxndb_total_df[rxndb_total_df['new_num'] > 0]
    print(rxndb_total_df.shape)
    rxndb_total_df['new_num'] = rxndb_total_df['reactant_inchikey0'].progress_apply(lambda x: sum([1 for i in x if i not in total_inchikey0 ]))
    rxndb_total_df = rxndb_total_df[rxndb_total_df['new_num'] == 1]
    print(rxndb_total_df.shape)
    rxndb_total_df['product_inchikey0'] = rxndb_total_df['product_inchikey0'].apply(lambda x: ('.').join(x))

    rxndb_total_df.to_csv(not_lipid_rxndb_inteme_only_one_new_path)

In [36]:
total_inchikey0 = load_pickle(total_met_inchikey0_file)
choose_only_one_new(not_lipid_rxndb_inteme_drop_duplicate_path,not_lipid_target_smiles_filter20percent_else_path,total_inchikey0,not_lipid_rxndb_inteme_only_one_new_path)

(978830, 11)
(978086, 12)
(3131, 12)
(492, 12)


In [37]:
def filter_df(df):
    for index, row in df.iterrows():
        reactant_smile = row['reactant_smile'].split('.')
        for i in reactant_smile:
            if not Chem.MolFromSmiles(i):
                df.drop(index, inplace=True)
                break
    for index, row in df.iterrows():
        productsmile = row['productsmile'].split('.')
        for i in productsmile:
            if not Chem.MolFromSmiles(i):
                df.drop(index, inplace=True)
                break
    return df

In [38]:
not_lipid_rxndb_inteme_only_one_new = pd.read_csv(not_lipid_rxndb_inteme_only_one_new_path)
print(not_lipid_rxndb_inteme_only_one_new.shape)

(492, 13)


In [39]:
chunks = np.array_split(not_lipid_rxndb_inteme_only_one_new, 50)
with mp.Pool(50) as pool:
    results = list(tqdm(pool.imap(filter_df, chunks), total=len(chunks)))
rxndb_total_df = pd.concat(results)
rxndb_total_df.shape

100%|██████████| 50/50 [00:00<00:00, 1247.73it/s]


(492, 13)

In [40]:
rxndb_total_df.to_csv(not_lipid_rxndb_inteme_only_one_new_target_path,index=False)

In [41]:
not_lipid_rxndb_inteme_drop_duplica = pd.read_csv(not_lipid_rxndb_inteme_drop_duplicate_path)
not_lipid_rxndb_inteme_drop_duplica.shape

(978830, 11)